In [1]:
import transformers
import torch 
import gc 
from tokens import * 

/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
gc.collect()
torch.cuda.empty_cache()

In [4]:
# Load the model and tokenizer with the access token
model = transformers.AutoModelForCausalLM.from_pretrained(
    mistral_model_id,
    use_auth_token=mistral_access_token,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=access_token
)

# Create the pipeline with the specified model and tokenizer
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)




/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [03:33<00:00, 71.06s/it]
/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
2024-10-24 16:14:23.811985: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical

In [5]:
def repeater(question):
  # Define messages
#   messages = [
#       {"role": "system", "content": "You are an AI assistant tasked with producing neutral news articles based on the provided topics. Ensure the content is unbiased, factual, and informative, and make sure to conclude the article logically."},
#       {"role": "user", "content": question}
#   ]
  messages = [
      {"role": "system", "content": "You are an AI assistant tasked with producing NEUTRAL news article based on the provided article information."
                                    "- Try to filter the political bias that maybe existent in the article"
                                    "- Don't create new information that is not existent in the article" 
                                    "- Write a new article as a journalist, ensuring the content is unbiased, factual, and informative."},
                                    
      {"role": "user", "content": question}
  ]
  # Apply chat template to messages
  prompt = pipeline.tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )

  # Define terminators
  terminators = [
      pipeline.tokenizer.eos_token_id,
  ]

  # Generate text
  outputs = pipeline(
      prompt,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.5,
      top_p=0.9,
      max_new_tokens=2024*2
  )

  with open('mistral_output.txt', 'w') as file: 
    file.write(outputs[0]["generated_text"][len(prompt):])

  del outputs
  gc.collect()
  torch.cuda.empty_cache()



In [6]:
with open('output.txt','r') as f:
  content = f.read()


In [7]:
repeater(content)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [21]:
del model